In [1]:
DATAPATH = '../midi/rag-split'

In [2]:
!echo $DATAPATH

../midi/rag-split


In [10]:
import os
import shutil
import hashlib

def split_midi_files(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Create subfolders for each split
    for split in '0123456789abcdef':
        os.makedirs(os.path.join(output_folder, split), exist_ok=True)
    
    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.mid') or filename.endswith('.midi'):
            file_path = os.path.join(input_folder, filename)
            
            # Calculate MD5 hash of the file
            with open(file_path, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            
            # Determine which subfolder to put the file in
            subfolder = file_hash[0]
            
            # Copy the file to the appropriate subfolder
            destination = os.path.join(output_folder, subfolder, filename)
            shutil.copy2(file_path, destination)
    
    print(f"Files have been split into subfolders in {output_folder}")

# Usage
input_folder = "../midi/rag"
output_folder = "../midi/rag-split"
split_midi_files(input_folder, output_folder)

Files have been split into subfolders in ../midi/rag-split


In [11]:
!python ./anticipation/train/midi-preprocess.py $DATAPATH

Preprocessing 152 files with 16 workers
Preprocess: 100%|█████████████████████████████| 152/152 [00:02<00:00, 67.03it/s]
Successfully processed 152 files (discarded 0.0%)


In [12]:
!python ./anticipation/train/tokenize-lakh.py $DATAPATH --augment 1

Tokenizing LakhMIDI
  encoding type: arrival
  train split: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd']
  validation split: ['e']
  test split: ['f']
Tokenization parameters:
  anticipation interval = 5s
  augment = 1x
  max track length = 3600s
  min track length = 10s
  min track events = 100






#5:   0%|                                                 | 0/7 [00:00<?, ?it/s]










#10:   0%|                                                | 0/9 [00:00<?, ?it/s]











#0:   0%|                                                | 0/10 [00:00<?, ?it/s]




#4:   0%|                                                | 0/10 [00:00<?, ?it/s]

#1:   0%|                                                | 0/11 [00:00<?, ?it/s]









#9:   0%|                                                | 0/16 [00:00<?, ?it/s]








#8:   0%|                                                 | 0/5 [00:00<?, ?it/s]






#6:   0%|                                              

In [17]:
!mv $DATAPATH/tokenized-events-e.txt $DATAPATH/valid.txt
!mv $DATAPATH/tokenized-events-f.txt $DATAPATH/test.txt
!cat $DATAPATH/tokenized-events-*.txt > $DATAPATH/train-ordered.txt

In [20]:
# I had to run `brew install coreutils` to get this to work
!shuf $DATAPATH/train-ordered.txt > $DATAPATH/train.txt

### Cleanup on this

In [21]:
!rm $DATAPATH/*/*.txt
!rm $DATAPATH/tokenized-events-*.txt
!rm $DATAPATH/train-ordered.txt